In [ ]:


from IPython import display

import time
from lib_xml_tree import *
from lib_connection import *
from lib_config_loader import load_config


    
if __name__ == '__main__':
    
    # Load IP, PORT configuration from config
    config = load_config()
    target_address = config['clients']['robot']['target_address']  
    target_port = config['clients']['robot']['target_port']

    
  
    UDPClientSocket = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
    server_address =  (target_address, target_port) 

    telegram = create_xml_fast([{'IPOC': '0'}])
    print('start_mess = :', telegram)


    # init dependant telegram variables
    watchDog_in = '0'
    
    while True:
        ################################################
        # SENDING MESSAGES TO ROBOT SERVER-SUBPROCESS
        ################################################  
        TransmitData = telegram
        SendData(TransmitData, UDPClientSocket, server_address)


        ################################################
        # RECEIVING MESSAGES FROM ROBOT SERVER-SUBPROCESS
        ################################################ 
        bytesAddressPair = UDPClientSocket.recvfrom(4096)

        ReceivedMessage = bytesAddressPair[0]
        address = bytesAddressPair[1]
        # clientMsg = "Message from Client:{}".format(ReceivedMessage)
        # clientIP = "Client IP Address:{}".format(address)
        #print(clientMsg)
        #print(clientIP)
        received_dict = extract_xml(ReceivedMessage)
        print('robot_received_telegram',ReceivedMessage)

        # use received data from cam
        try:
            watchDog_in = received_dict['WatchDog_in']
        except Exception as ex:
            print(f" Data can not be extracted from received to robot telegram: {ex}")        



        ################################################
        # PREPARING THE MESSAGE TO BE SENT
        ################################################ 
        sent_mess_list = list() # List of dictionaries. every dictionary will be the element of ElementTree
        sent_mess_list.append({"Sen" : {'Type' : 'KRC'}})  
        sent_mess_list.append({'WatchDog_out': str(1 - int(watchDog_in))})                        
        sent_mess_list.append({'Frame_assigned': '0'})
        sent_mess_list.append({'Position_reached': '0'})
        sent_mess_list.append({'Need_cam_cal': '0'})        
        sent_mess_list.append({'Screenshot': '0'}) 
        sent_mess_list.append({'IPOC': str(int(received_dict['IPOC']) + 1)})

        #cresting a new telegram to send
        # telegram = create_xml(sent_mess_list)
        telegram = create_xml_fast(sent_mess_list)
        print('robot_sending_telegram',  telegram)
        
                
        display.clear_output(wait=True)

        


        
        

robot_received_telegram b'<Sen Type="ImFree"><XYZ1 X="0.0" Y="0.0" Z="0.0" /><XYZ2 X="0.0" Y="0.0" Z="0.0" /><XYZ3 X="0.0" Y="0.0" Z="0.0" /><XYZ4 X="0.0" Y="0.0" Z="0.0" /><CAM_CAL_RES X="0.0" Y="0.0" Z="0.0" A="0.0" B="0.0" C="0.0" /><WatchDog_in>0</WatchDog_in><Frame_assign>0</Frame_assign><Move_next_pt>0</Move_next_pt><Cam_cal_in_proc>0</Cam_cal_in_proc><See_4_targets>0</See_4_targets><IPOC>5619</IPOC></Sen>'
robot_sending_telegram b'<Sen Type="KRC"><WatchDog_out>1</WatchDog_out><Frame_assigned>0</Frame_assigned><Position_reached>0</Position_reached><Need_cam_cal>0</Need_cam_cal><Screenshot>0</Screenshot><IPOC>5620</IPOC></Sen>'


In [ ]:
import socket
import io
import xml.etree.ElementTree as xml

In [ ]:
#for test only 
# Creates structure for ambedding into the xml package, which will be send to robot
def create_Rkorr_for_sending(frame = {'X' : 0, 'Y' : 0, 'Z' : 0}):  #frame = {'X' : float, 'Y' : float, 'Z' : float, 'A' : float, 'B' : float, 'C' : float}
    # print(frame)
    Rkorr = {'X' : str(frame['X']), 
             'Y' : str(frame['Y']), 
             'Z' : str(frame['Z'])}
    
    return Rkorr


def create_xml(target_frame_1 = {'X' : 0.100, 'Y' : 1.000, 'Z' : 0.030, 'A' : 0.000, 'B' : 0.000, 'C' : 0.000},
              target_frame_2 = {'X' : 0, 'Y' : 0, 'Z' : 0},
              target_frame_3 = {'X' : 0, 'Y' : 0, 'Z' : 0},
              target_frame_4 = {'X' : 0, 'Y' : 0, 'Z' : 0},
              IPOCtimerReseived = '123456789'):

    SenAttr = dict(Type='ImFree')

    root = xml.Element("Sen", SenAttr)
    
    ################# RESTORE IT ##################
    RkorrArrt_1 = create_Rkorr_for_sending(target_frame_1)
    Rkorr_1 = xml.SubElement(root, "RKorr", RkorrArrt_1)
    ################################################
    #print(RkorrArrt_1)
    #print(Rkorr_1)





    #RkorrArrt = dict(X='0.03')
    #Rkorr = xml.SubElement(root, "RKorr", RkorrArrt)
    #print(RkorrArrt) 
   




    
#     RkorrArrt_2 = create_Rkorr_for_sending(target_frame_2)
#     Rkorr_2 = xml.SubElement(root, "Rkorr", RkorrArrt_2)
    
#     RkorrArrt_3 = create_Rkorr_for_sending(target_frame_3)
#     Rkorr_3 = xml.SubElement(root, "Rkorr", RkorrArrt_3)
    
#     RkorrArrt_4 = create_Rkorr_for_sending(target_frame_4)
#     Rkorr_4 = xml.SubElement(root, "Rkorr", RkorrArrt_4)
    
    
    IPOC = xml.SubElement(root, "IPOC")
    IPOC.text = IPOCtimerReseived
    # IPOC.text = '123456789'
    
    
    tree = xml.ElementTree(root)
    
    
    DecodedTree = xml.tostring(root, encoding='utf8', method='xml', short_empty_elements='true', xml_declaration=False)
    return(DecodedTree)



In [ ]:
xml_parcel = create_xml()
xml_parcel

In [ ]:
d = {'XYZ1' : '123'}
